## EXPERIMENT

1. multivariate imputation for age 
2. include cabin (find a way to impute and find the missing values tho)
3. try logistic regression
4. try random forests 

In [ ]:
import pandas as pd 

In [5]:
filepath = '/Users/thananpornsethjinda/Desktop/internship/ml/machine-learning-journey-log/model-training/classification/titanic/data/train.csv'

raw_training_data = pd.read_csv(filepath)

raw_training_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
filepath = '/Users/thananpornsethjinda/Desktop/internship/ml/machine-learning-journey-log/model-training/classification/titanic/data/test.csv'

raw_test_data = pd.read_csv(filepath)

raw_test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [7]:
from basic_cleaning import clean_dataframe

clean_train_data = clean_dataframe(raw_training_data)

print('-' * 50)

clean_test_data = clean_dataframe(raw_test_data)

[INFO] Standardized column names.
[INFO] Standardized string columns (lowercase + trimmed).
[INFO] Missing values found in columns:
age         177
cabin       687
embarked      2
dtype: int64
[INFO] High-cardinality columns (consider encoding strategies): ['name', 'ticket', 'cabin']
[INFO] Potential numeric outliers detected:
{'age': np.int64(11), 'sibsp': np.int64(46), 'parch': np.int64(213), 'fare': np.int64(116)}
[INFO] Converted suitable object columns to category dtype.
[INFO] Data cleaning complete.
--------------------------------------------------
[INFO] Standardized column names.
[INFO] Standardized string columns (lowercase + trimmed).
[INFO] Missing values found in columns:
age       86
fare       1
cabin    327
dtype: int64
[INFO] High-cardinality columns (consider encoding strategies): ['name', 'ticket']
[INFO] Potential numeric outliers detected:
{'age': np.int64(2), 'sibsp': np.int64(11), 'parch': np.int64(94), 'fare': np.int64(55)}
[INFO] Converted suitable object colu

In [8]:
clean_train_data

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"braund, mr. owen harris",male,22.0,1,0,a/5 21171,7.2500,NaN,s
1,2,1,1,"cumings, mrs. john bradley (florence briggs th...",female,38.0,1,0,pc 17599,71.2833,c85,c
2,3,1,3,"heikkinen, miss. laina",female,26.0,0,0,ston/o2. 3101282,7.9250,NaN,s
3,4,1,1,"futrelle, mrs. jacques heath (lily may peel)",female,35.0,1,0,113803,53.1000,c123,s
4,5,0,3,"allen, mr. william henry",male,35.0,0,0,373450,8.0500,NaN,s
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"montvila, rev. juozas",male,27.0,0,0,211536,13.0000,NaN,s
887,888,1,1,"graham, miss. margaret edith",female,19.0,0,0,112053,30.0000,b42,s
888,889,0,3,"johnston, miss. catherine helen ""carrie""",female,NaN,1,2,w./c. 6607,23.4500,NaN,s
889,890,1,1,"behr, mr. karl howell",male,26.0,0,0,111369,30.0000,c148,c


In [48]:
clean_train_data_modified = clean_train_data.drop(['ticket', 'fare', 'cabin', 'name', 'passengerid','sibsp', 'parch'], axis=1)

clean_train_data_modified

,survived,pclass,sex,age,embarked
0,0,3,male,22.0,s
1,1,1,female,38.0,c
2,1,3,female,26.0,s
3,1,1,female,35.0,s
4,0,3,male,35.0,s
...,...,...,...,...,...
886,0,2,male,27.0,s
887,1,1,female,19.0,s
888,0,3,female,NaN,s
889,1,1,male,26.0,c


In [49]:
clean_test_data_modified = clean_test_data.drop(['ticket', 'fare', 'cabin', 'name', 'passengerid','sibsp', 'parch'], axis=1)

clean_test_data_modified

,pclass,sex,age,embarked
0,3,male,34.5,q
1,3,female,47.0,s
2,2,male,62.0,q
3,3,male,27.0,s
4,3,female,22.0,s
...,...,...,...,...
413,3,male,NaN,s
414,1,female,39.0,c
415,3,male,38.5,s
416,3,male,NaN,s


In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='constant', fill_value='unknown')

clean_train_data_modified[['embarked']] = imputer.fit_transform(clean_train_data_modified[['embarked']])

clean_train_data_modified['embarked'].isnull().sum()


np.int64(0)

In [55]:
categorical_columns = ['sex', 'embarked']

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()

enc.fit(clean_train_data_modified[categorical_columns])

encoded_columns = list(enc.get_feature_names_out(categorical_columns))

clean_train_data_modified[encoded_columns] = enc.transform(clean_train_data_modified[categorical_columns]).toarray()

clean_test_data_modified[encoded_columns] = enc.transform(clean_test_data_modified[categorical_columns]).toarray()

clean_train_data_modified

,survived,pclass,sex,age,embarked,sex_female,sex_male,embarked_c,embarked_q,embarked_s,embarked_unknown
0,0,3,male,22.0,s,0.0,1.0,0.0,0.0,1.0,0.0
1,1,1,female,38.0,c,1.0,0.0,1.0,0.0,0.0,0.0
2,1,3,female,26.0,s,1.0,0.0,0.0,0.0,1.0,0.0
3,1,1,female,35.0,s,1.0,0.0,0.0,0.0,1.0,0.0
4,0,3,male,35.0,s,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,s,0.0,1.0,0.0,0.0,1.0,0.0
887,1,1,female,19.0,s,1.0,0.0,0.0,0.0,1.0,0.0
888,0,3,female,NaN,s,1.0,0.0,0.0,0.0,1.0,0.0
889,1,1,male,26.0,c,0.0,1.0,1.0,0.0,0.0,0.0


In [78]:
X_train = clean_train_data_modified[['pclass', 'age'] + encoded_columns]

y_train = clean_train_data_modified['survived']

X_test = clean_test_data_modified[['pclass', 'age'] + encoded_columns]

In [ ]:
from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import IterativeImputer

from sklearn.linear_model import BayesianRidge

imp = IterativeImputer(estimator=BayesianRidge(), max_iter=10, random_state=42)

imputed_array_train = imp.fit_transform(X_train)

imputed_array_test = imp.transform(X_test)

X_train_imputed = pd.DataFrame(imputed_array_train, columns=X_train.columns)

X_test_imputed = pd.DataFrame(imputed_array_test, columns=X_test.columns)

## Training using random forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

rf_model.fit(X_train_imputed, y_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [83]:
y_preds = rf_model.predict(X_train_imputed)

In [87]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train, y_preds)

array([[518,  31],
       [ 63, 279]])

In [88]:
test_preds = rf_model.predict(X_test_imputed)

In [89]:
output = pd.DataFrame({'PassengerId': raw_test_data.PassengerId, 'Survived': test_preds})

output


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [90]:
output.to_csv('submission2.csv', index=False)
